In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [129]:
rank_data=pd.read_csv('../data.csv')
rank_data = rank_data.drop(columns=['Avg_KDA']) # drop Avg_KDA to avoid multicolinearity
rank_data

,Avg_Kills,Avg_Deaths,Avg_Assists,Avg_total_gold,Avg_total_minion,Rank
0,9.92,4.15,7.31,12732.31,114.54,0
1,4.15,4.85,6.50,9208.70,131.40,0
2,6.50,9.00,8.50,12305.95,164.05,0
3,6.70,5.95,6.55,11465.35,152.70,0
4,10.25,7.90,6.15,12844.50,155.60,0
...,...,...,...,...,...,...
866,7.60,6.85,8.35,11153.65,33.45,8
867,6.45,6.00,8.00,11494.40,172.30,8
868,5.90,5.65,5.90,10457.45,124.25,8
869,7.50,4.90,8.00,11815.15,182.20,8


In [130]:
X=rank_data.iloc[:,:5].to_numpy() # X takes the first 5 features - columns
y=rank_data.iloc[:,-1].to_numpy() # y is the label - the last column

In [131]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, shuffle = True, random_state = 42, stratify = y , test_size=0.2)

In [ ]:
# For deployment - clipping input
X_min = X_train.min(axis=0) # Take the min value of every columns
X_max = X_train.max(axis=0) # Take the max value of every columns
print(X_min)
print(X_max)
np.save('data_min.npy', X_min)
np.save('data_max.npy', X_max)

print("Saved data!")

[9.0000e-01 2.8000e+00 3.5000e+00 6.7381e+03 1.5800e+01]
[1.33000e+01 1.04500e+01 1.90500e+01 1.55536e+04 2.21550e+02]
Saved data!


In [83]:
# Preprocess
train_mean = np.mean(X_train, axis = 0)
train_std = np.std(X_train, axis = 0)
X_train = (X_train - train_mean) / train_std   # Transform X = (X - muy)/sigma holds [0, 1]

# When a new point sent to the model, there's no such a new dataset for test yet
# Must use from what's been learnt to process
X_test = (X_test - train_mean) / train_std  

print(f'X_train: {X_train}\n\n')
print(f'X_test: {X_test}')

X_train: [[-0.81531008  0.51720542  0.66100031  0.0430277   0.2253418 ]
 [ 1.50241924 -0.20879709 -0.48590363  1.08551027  1.13907225]
 [ 0.39085517  2.16026374 -0.29151313  0.85538548  0.10440688]
 ...
 [-0.93356158 -1.16406356 -0.29151313 -0.6696478   0.15367666]
 [-2.09242624 -0.667325    1.36080611 -1.88118333 -1.28141764]
 [-1.64780062  0.48663689  0.25278027 -2.16819527 -1.36132426]]


X_test: [[-8.21508094e-02 -8.20167631e-01  7.77634613e-01 -5.02837773e-01
  -7.79761703e-01]
 [ 9.11161757e-01  2.19847440e+00  3.30536466e-01  1.03526860e+00
   3.99082639e-02]
 [-1.95052445e+00 -9.73010266e-01  1.03034226e+00 -2.35902389e+00
  -1.06463061e+00]
 [ 6.98309065e-01 -8.58378290e-01  2.33341216e-01  6.18403738e-01
   9.97533614e-01]
 [-3.42304101e-01 -1.20227422e+00  3.89507170e-02 -1.55279499e-01
  -9.17717086e-01]
 [ 5.97509858e-02 -1.77438001e-02 -8.55245579e-01  7.60308330e-02
   1.01813734e+00]
 [ 7.45609663e-01  1.66352518e+00  2.52780266e-01  1.75717216e+00
   3.12235772e-01]
 [

In [84]:
print(X_train.shape,y_train.shape, X_test.shape,y_test.shape)

(696, 5) (696,) (175, 5) (175,)


Denoted vecto theta=[theta1 theta2 ... thetaN] as W

First, we build a model that use a linear function - SGD

In [88]:
def calculate_y_hat(W,X,b):
    m=X.shape[0]
    y_hat=np.zeros(m)
    y_hat=np.dot(X,W)+b # Linear regression
    return y_hat

In [ ]:
def calculate_loss_function(y_hat,y):
    '''
    - Here we use MSE instead of other classification loss function because we want to see how close the ability of player is
    between ranks (ordinal scale). For instance: y_hat = 0.9 => still iron but very close to silver => potential silver
    - MSE punishes stronger than MAE - "Square" vs "Absolute" => MSE
    '''
    m = y_hat.shape[0]
    loss=0
    for i in range(m):
        loss+=((y_hat[i]-y[i])**2)/m/2 # we add /2 so after derivaties, it cancels out number '2'
    return loss

In [90]:
def calculate_gradient_vector(W,X,b,Y):
    m=X.shape[0]
    n=W.shape[0]
    dW=np.zeros(n)
    db=0
    y_hat=calculate_y_hat(W,X,b)
    for i in range(m):
        for j in range(n):
            dW[j]+=(y_hat[i]-Y[i])*X[i][j]/m
        db+=(y_hat[i]-Y[i])/m
    return dW,db

In [91]:
W = np.random.rand(*X.shape[1:]) * 0.01 # break symmetry
b = np.random.rand()
loss = 0

In [92]:
learning_rate = 0.001
iterate_time = 10000

In [93]:
for k in range(iterate_time):
    dW,db=calculate_gradient_vector(W, X_train,b,y_train)

    W-=learning_rate*dW
    b-=learning_rate*db

    y_hat=calculate_y_hat(W,X_train,b)
    loss=calculate_loss_function(y_hat, y_train)
    
    if (k+1) % 100 ==0:
        print(f"Cost {k+1}: {loss}")

Cost 100: 7.193393448732746
Cost 200: 6.356119117470482
Cost 300: 5.675597786790086
Cost 400: 5.121537695088149
Cost 500: 4.669857547519011
Cost 600: 4.301276738988022
Cost 700: 4.000275311138349
Cost 800: 3.7543102539082405
Cost 900: 3.5532142422935618
Cost 1000: 3.3887274775841187
Cost 1100: 3.254128854093166
Cost 1200: 3.1439427058840668
Cost 1300: 3.0537040083609543
Cost 1400: 2.979769387622222
Cost 1500: 2.919164399491132
Cost 1600: 2.869459755660091
Cost 1700: 2.8286707927154917
Cost 1800: 2.7951756886329604
Cost 1900: 2.7676488518841755
Cost 2000: 2.745006620683053
Cost 2100: 2.726362968382935
Cost 2200: 2.71099335334654
Cost 2300: 2.698305204668822
Cost 2400: 2.687813818629535
Cost 2500: 2.6791226694100008
Cost 2600: 2.67190732264778
Cost 2700: 2.6659022905125753
Cost 2800: 2.6608902889816766
Cost 2900: 2.6566934572718317
Cost 3000: 2.653166180256715
Cost 3100: 2.650189220626324
Cost 3200: 2.64766492131741
Cost 3300: 2.6455132826221766
Cost 3400: 2.643668754197277
Cost 3500: 2.

In [121]:
y_pred_test_1 = calculate_y_hat(W,X_test,b)
y_pred_test_1

array([5.05120599, 1.34246105, 5.77148034, 4.85642699, 5.1925681 ,
       4.23433207, 1.41104107, 5.10009502, 2.9205047 , 3.18088922,
       3.37117507, 3.35835422, 1.0008349 , 5.717795  , 4.94884058,
       1.61866313, 4.37442627, 6.10599349, 3.55476762, 5.281104  ,
       3.64029082, 5.43802218, 2.9468311 , 3.46951161, 4.64098831,
       3.62189446, 3.90354517, 6.52200685, 3.2193121 , 4.6595265 ,
       5.27628706, 4.5204159 , 2.48787544, 3.23123124, 3.47290288,
       3.45568664, 2.68544489, 3.96527383, 3.3066928 , 4.43060719,
       3.30805531, 4.38979909, 2.28153616, 4.83955009, 4.31612725,
       1.63242069, 5.15614808, 4.23655976, 4.38733679, 4.61872716,
       5.53997658, 4.30501837, 4.9981773 , 4.75127666, 2.80444368,
       3.85415807, 4.88633728, 4.13198111, 4.2683827 , 2.75923706,
       4.07891531, 5.47684664, 3.44695924, 5.41539551, 2.50277925,
       5.44736284, 3.26626518, 2.30335714, 0.583852  , 2.70401907,
       3.14265356, 4.5874417 , 2.38515006, 4.04203314, 5.60300

In [95]:
y_test

array([8, 0, 5, 4, 7, 6, 2, 6, 4, 3, 3, 8, 2, 8, 2, 4, 5, 8, 6, 8, 5, 8,
       4, 1, 7, 5, 6, 5, 2, 7, 0, 4, 1, 6, 8, 6, 1, 5, 7, 0, 4, 6, 3, 8,
       5, 2, 3, 2, 7, 6, 6, 6, 0, 8, 0, 2, 3, 0, 6, 1, 1, 2, 1, 3, 1, 6,
       1, 5, 0, 0, 3, 3, 0, 7, 8, 4, 1, 3, 3, 1, 7, 6, 4, 5, 7, 5, 7, 8,
       4, 0, 8, 7, 4, 0, 7, 7, 1, 4, 8, 3, 2, 1, 6, 0, 1, 0, 5, 8, 7, 2,
       5, 1, 4, 1, 2, 3, 0, 2, 7, 6, 4, 6, 3, 6, 0, 6, 6, 7, 7, 3, 5, 1,
       1, 4, 3, 3, 3, 3, 5, 4, 5, 5, 0, 5, 2, 8, 7, 0, 4, 0, 8, 4, 1, 8,
       7, 1, 1, 3, 8, 6, 0, 7, 5, 4, 2, 5, 0, 2, 8, 3, 5, 4, 8, 4, 7])

In [ ]:
def calculate_r_squared(y_true, y_pred):
    # Tổng bình phương sai số của mô hình (Residual Sum of Squares)
    rss = np.sum((y_true - y_pred)**2)
    # Tổng bình phương sai số so với trung bình (Total Sum of Squares)
    tss = np.sum((y_true - np.mean(y_true))**2)
    
    r2 = 1 - (rss / tss)
    return r2

In [123]:
y_pred_train_1 = calculate_y_hat(W,X_train,b)
mse1_train = np.mean((y_train-y_pred_train_1)**2)
mse1_test = np.mean((y_test-y_pred_test_1)**2)

print (f'MSE of linear regression on train set: {mse1_train}')
print (f'MSE of linear regression on test set: {mse1_test}')
print(f'R-square: {calculate_r_squared(y_test, y_pred_test_1)}')
# Can explain ~ 22% what's going on the data

MSE of linear regression on train set: 5.239500484908423
MSE of linear regression on test set: 5.255545789071955
R-square: 0.22019820838019066


In [101]:
# Let's have a look on the parameters
print(W)
print(b)

[ 0.3229861  -1.0444049  -0.09494827 -0.65721067  0.11165735]
4.051583296001721


Now we do the regression with a **quadratic function** (Polynomial regression, $deg$ = 2), with $\theta$ equals to:

$$\theta = (\Phi^T \Phi)^{-1} \Phi^T Y$$

$$\Phi = \begin{bmatrix} 1 & x_1 & x_1^2 \\ 1 & x_2 & x_2^2 \\ \vdots & \vdots & \vdots \\ 1 & x_n & x_n^2 \end{bmatrix}$$

In [118]:
X_design_2_train = [np.ones(X_train.shape[0])]
# Add deg 1
for i in range(X_train.shape[1]):
    X_design_2_train.append(X_train[:, i])
# Add deg 2
for i in range(X_train.shape[1]):
    for j in range(i, X_train.shape[1]): 
        X_design_2_train.append(X_train[:, i] * X_train[:, j])

X_design_2_train = np.column_stack(X_design_2_train)
beta_2 = np.linalg.inv(X_design_2_train.T @ X_design_2_train) @ X_design_2_train.T @ y_train

y_pred_train_2 = X_design_2_train @ beta_2

# Predict and test
X_design_2_test = [np.ones(X_test.shape[0])]
for i in range(X_test.shape[1]):
    X_design_2_test.append(X_test[:, i])
for i in range(X_test.shape[1]):
    for j in range(i, X_test.shape[1]): 
        X_design_2_test.append(X_test[:, i] * X_test[:, j]) 

X_design_2_test = np.column_stack(X_design_2_test)
y_pred_test_2 = X_design_2_test @ beta_2

mse2_train = np.mean((y_train - y_pred_train_2) ** 2)
mse2_test = np.mean((y_test - y_pred_test_2) ** 2)
print("MSE on train set:", mse2_train)
print("MSE on test set:", mse2_test)
print("R-square:", calculate_r_squared(y_test, y_pred_test_2) )

MSE on train set: 4.170547021463015
MSE on test set: 4.131100482998185
R-square: 0.3870399598264562


Now we have $3-degree$ regression model with Ridge regularization

To capture even more complex interactions, we extend the model to the **third degree**. 
The parameters $\theta$ are estimated using the **Ridge Regression** formula:

$$\theta = ( \Phi^T \Phi + \lambda I )^{-1} \Phi^T Y$$
$$\Phi(\mathbf{x}) = [1, \underbrace{x_i}_{\text{Deg 1}}, \underbrace{x_i x_j}_{\text{Deg 2}}, \underbrace{x_i x_j x_k}_{\text{Deg 3}}]$$
Where:
- $\Phi$ is the augmented design matrix including cubic terms ($x_i x_j x_k$).
- $\lambda$ is the regularization strength
- $I$ is the identity matrix (with $I_{0,0} = 0$ to avoid penalizing the bias term).

In [117]:
X_design_train_3 = [np.ones(X_train.shape[0])]
# Add deg 1
for i in range(X_train.shape[1]):
    X_design_train_3.append(X_train[:, i])
# Add deg 2
for i in range(X_train.shape[1]):
    for j in range(i, X_train.shape[1]):
        X_design_train_3.append(X_train[:, i] * X_train[:, j])
# Add deg 3
for i in range(X_train.shape[1]):
    for j in range(i, X_train.shape[1]):
        for k in range(j, X_train.shape[1]):
            X_design_train_3.append(X_train[:, i] * X_train[:, j] * X_train[:, k])

X_design_train_3 = np.column_stack(X_design_train_3)

# Lambda (l2 penalty) - Ridge regression needed since deg 3 causes #features increase quickly
lambda_reg_3 = 5
I = np.identity(X_design_train_3.shape[1])
I[0, 0] = 0 # Usually we don't cancel out the bias

XTX_3 = X_design_train_3.T @ X_design_train_3
XTy_3 = X_design_train_3.T @ y_train

beta_3 = np.linalg.inv(XTX_3 + lambda_reg_3 * I) @ XTy_3
y_pred_train_3 = X_design_train_3 @ beta_3

# Predict and test
X_design_test_3 = [np.ones(X_test.shape[0])]
for i in range(X_test.shape[1]):
    X_design_test_3.append(X_test[:, i])
for i in range(X_test.shape[1]):
    for j in range(i, X_test.shape[1]):
        X_design_test_3.append(X_test[:, i] * X_test[:, j])
for i in range(X_test.shape[1]):
    for j in range(i, X_test.shape[1]):
        for k in range(j, X_test.shape[1]):
            X_design_test_3.append(X_test[:, i] * X_test[:, j] * X_test[:, k])

X_design_test_3 = np.column_stack(X_design_test_3)

# Predict and test
y_pred_test_3 = X_design_test_3 @ beta_3

mse_train_3 = np.mean((y_train - y_pred_train_3) ** 2)
mse_test_3 = np.mean((y_test - y_pred_test_3) ** 2)

print(f"MSE on train set: {mse_train_3}")
print(f"MSE on test set: {mse_test_3}")
print(f"R-square: {calculate_r_squared(y_test, y_pred_test_3)}")

MSE Train: 3.8710040123972997
MSE Test: 4.272606903987062
R-square Test: 0.36604367037498176


From the MSE of all model below, we believe that with the complexity of higher degree model, the data can be fit more than ever. We can say that quadratic regression model is the optimal choice here ($R^2$ $\approx$ 0.38; $ MSE_{test}$ $\approx$ 4.13)

In [127]:
# Save weights and bias on the whole dataset to deploy
X_design_final = [np.ones(X.shape[0])]
for i in range(X.shape[1]):
    X_design_final.append(X[:, i])
for i in range(X.shape[1]):
    for j in range(i, X.shape[1]):
        X_design_final.append(X[:, i] * X[:, j])
X_design_final = np.column_stack(X_design_final)

theta_final = np.linalg.pinv(X_design_final.T @ X_design_final) @ X_design_final.T @ y
print(theta_final)
np.save('model.npy', theta_final)
print("Model saved successfully!")

[ 2.51964301e-05 -2.56017048e-04  5.27056521e-05  8.10163661e-04
  1.30950222e-03  9.24688198e-04  2.98320456e-03 -5.79983197e-04
 -8.79864990e-03  1.13369685e-04 -7.38619749e-03  6.42071561e-04
  2.21255066e-03 -5.29638996e-05  5.28915144e-04  2.79390672e-02
 -4.49616577e-05  6.80554326e-04 -8.17828897e-08 -3.30893718e-06
  3.70786331e-04]
Model saved successfully!
